# __Delta-Hedging Simulations__

<br>

Finance 6470: Derivatives Markets <br>
Tyler J. Brough <br>
Last Update: April 22, 2019 <br>
<br>

In [21]:
import enum
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
plt.rcParams['figure.figsize'] = [20, 10]

In [10]:
def bsmCall(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    d2 = d1 - v * np.sqrt(t)
    prc = S * np.exp(-q * t) * norm.cdf(d1) - K * np.exp(-r * t) * norm.cdf(d2)
    
    return prc

In [11]:
def bsmPut(S,K,v,r,t,q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    d2 = d1 - v * np.sqrt(t)
    prc = K * np.exp(-r * t) * norm.cdf(-d2) - S * np.exp(-q * t) * norm.cdf(-d1)
    
    return prc

In [12]:
def bsmCallDelta(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    delta = np.exp(-q * t) * norm.cdf(d1) 
    
    return delta

In [13]:
def bsmPutDelta(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    delta = np.exp(-q * t) * norm.cdf(-d1) 
    
    return delta

In [14]:
def bsmGamma(S, K, v, r, t, q):
    d1 = ((np.log(S/K) + (r - q + 0.5 * v * v)*t) / (v * np.sqrt(t)))
    gamma = (np.exp(-q * t) * norm.pdf(d1)) / (S * v * np.sqrt(t))
    
    return gamma

In [68]:
## An example (see McDonald 3rd pp. 383, Table 13.1)
S = 40.0
K = 40.0
v = 0.30
r = 0.08
tau = 91/365
q = 0.0

callPrc = bsmCall(S, K, v, r, tau, q)
callDelta = bsmCallDelta(S, K, v, r, tau, q)
callGamma = bsmGamma(S, K, v, r, tau, q)
print(f"The Call Price: {callPrc : 0.4f}")
print(f"The Call Delta: {callDelta : 0.4f}")
print(f"The Call Gamma: {callGamma : 0.4f}")

The Call Price:  2.7804
The Call Delta:  0.5824
The Call Gamma:  0.0652


In [167]:
class MarketMaker(object):
    def __init__(self, spot, strike, vol, rate, expiry, div, pricer, delta):
        self.__spot = spot
        self.__strike = strike
        self.__vol = vol
        self.__rate = rate
        self.__expiry = expiry
        self.__dt = 1/365
        self.__div = div
        self.__pricer = pricer
        self.__delta = delta
        self.__units = 100.0
        self.__option = self.__units * self.__pricer(self.__spot, self.__strike, self.__vol, self.__rate, self.__expiry, self.__div)
        self.__option_delta = self.__units * self.__delta(self.__spot, self.__strike, self.__vol, self.__rate, self.__expiry, self.__div)
        self.__market_value = self.__option_delta * self.__spot - self.__option
        self.__interest = 0.0
        self.__capital_gain = 0.0
        self.__profit = 0.0
              
    def show(self):
        print("Stock ($): \t\t{0:.2f}".format(self.__spot))
        print("Option ($): \t\t{0:.2f}".format(self.__option))
        print("100 x Delta: \t\t{0:.2f}".format(self.__option_delta))
        print("Investment ($): \t{0:0.2f}".format(self.__market_value))
        print("Interest ($): \t\t{0:0.2f}".format(self.__interest))
        print("Capital Gain ($): \t{0:0.2f}".format(self.__capital_gain))
        print("Daily Profit ($): \t{0:0.2f}".format(self.__profit))
        print("\n")

    def update(self, spot):
        mktVal = self.__market_value
        optDelta = self.__option_delta
        self.__interest = -(np.exp(self.__rate / 365) - 1.0) * mktVal
        self.__spot = spot
        self.__expiry -= self.__dt
        self.__option = self.__units * self.__pricer(self.__spot, self.__strike, self.__vol, self.__rate, self.__expiry, self.__div)
        self.__option_delta = self.__units * self.__delta(self.__spot, self.__strike, self.__vol, self.__rate, self.__expiry, self.__div)
        self.__market_value = self.__option_delta * self.__spot - self.__option
        self.__capital_gain = self.__market_value - mktVal - (spot * (self.__option_delta - optDelta))
        self.__profit = self.__market_value - mktVal - (spot * (self.__option_delta - optDelta)) + self.__interest

In [169]:
## See McDonald 3rd pp. 386, Table 13.2
spot_t = [40.50, 39.25, 38.75, 40.0, 40.0]
mm = MarketMaker(S, K, v, r, tau, q, bsmCall, bsmCallDelta)

print("Day t={0:d}".format(0))
print("-------------------------------")
mm.show()

for t, spot in enumerate(spot_t):
    mm.update(spot)
    print("Day t={0:d}".format(t+1))
    print("-------------------------------")
    mm.show()

Day t=0
-------------------------------
Stock ($): 		40.00
Option ($): 		278.04
100 x Delta: 		58.24
Investment ($): 	2051.58
Interest ($): 		0.00
Capital Gain ($): 	0.00
Daily Profit ($): 	0.00


Day t=1
-------------------------------
Stock ($): 		40.50
Option ($): 		306.21
100 x Delta: 		61.42
Investment ($): 	2181.31
Interest ($): 		-0.45
Capital Gain ($): 	0.95
Daily Profit ($): 	0.50


Day t=2
-------------------------------
Stock ($): 		39.25
Option ($): 		232.82
100 x Delta: 		53.11
Investment ($): 	1851.65
Interest ($): 		-0.48
Capital Gain ($): 	-3.39
Daily Profit ($): 	-3.87


Day t=3
-------------------------------
Stock ($): 		38.75
Option ($): 		205.46
100 x Delta: 		49.56
Investment ($): 	1715.12
Interest ($): 		-0.41
Capital Gain ($): 	0.81
Daily Profit ($): 	0.40


Day t=4
-------------------------------
Stock ($): 		40.00
Option ($): 		271.04
100 x Delta: 		58.06
Investment ($): 	2051.35
Interest ($): 		-0.38
Capital Gain ($): 	-3.62
Daily Profit ($): 	-4.00


Day t=5